# Introduction

The `swarmauri` toolkit provides a powerful set of prebuilt tools that can be used to perform various tasks like text analysis, weather fetching, and other utilities.

The toolkit consists of tools that are already defined and registered with the `swarmauri` package. Each tool is a subclass of `ToolBase` and requires specific parameters to function properly.

Tools are registered using the `SubclassUnion.update()` method, which allows them to be discovered by the toolkit system. However, in most cases, you do not need to manually register tools since they are automatically registered when imported.

If you need to register a new tool or update an existing one, you can do so like this:


In [4]:
from swarmauri_core.typing import SubclassUnion
from swarmauri.tools.base.ToolBase import ToolBase

from swarmauri.tools.concrete import WeatherTool
SubclassUnion.update(baseclass=ToolBase, type_name="WeatherTool", obj=WeatherTool)

# Why Customize?

In the previous notebook we saw:

In [5]:
# Instantiate the WeatherTool
weather_tool = WeatherTool()

# Fetch weather for a specific location
weather_info = weather_tool(location="London", unit="celsius")
print(weather_info)

{'weather_info': "('London', 'celsius')"}


The output you are seeing indicates that the `WeatherTool` is returning a string representation of the tuple with the location and `unit` parameters, rather than actual weather data.

This suggests that the `__call__` method in your WeatherTool class is currently just returning a tuple of the parameters, but it isn't yet implemented to fetch actual weather data.

To fix this, you would need to modify the `__call__` method of the WeatherTool class to actually fetch the weather information using a weather API or any other method, instead of just returning the location and unit as a string.

# Modifying the WeatherTool Class to Fetch Weather Data

To ensure that your `WeatherTool` fetches actual weather data rather than just returning the location and unit, you'll need to modify the `__call__` method of your `WeatherTool` class.

Let's assume you are using the OpenWeatherMap API for this. Here's how you can modify the `WeatherTool` class to fetch actual weather data:

- Install requests library (if not already installed):
```bash
pip install requests
```
## Update `WeatherTool` class to fetch weather data:

In [19]:
import requests
from pydantic import Field
from typing import List, Literal, Dict
from swarmauri.tools.concrete.Parameter import Parameter

In [21]:
# Fetch environment variables
import os
from dotenv import load_dotenv

load_dotenv()
# Fetch the openweather API Key
OPENWEATHER_API_KEY = os.getenv("OPENWEATHER_API_KEY")

In [22]:
class WeatherTool(ToolBase):
    version: str = "0.1.dev1"
    parameters: List[Parameter] = Field(
        default_factory=lambda: [
            Parameter(
                name="location",
                type="string",
                description="The location for which to fetch weather information",
                required=True,
            ),
            Parameter(
                name="unit",
                type="string",
                description="The unit for temperature ('fahrenheit' or 'celsius')",
                required=True,
                enum=["fahrenheit", "celsius"],
            ),
        ]
    )
    name: str = "WeatherTool"
    description: str = "Fetch current weather info for a location"
    type: Literal["WeatherTool"] = "WeatherTool"

    def __call__(self, location, unit="fahrenheit") -> Dict[str, str]:
        # OpenWeatherMap API key (replace with your actual API key)
        api_key = OPENWEATHER_API_KEY
        
        # Construct the API request URL
        url = f"http://api.openweathermap.org/data/2.5/weather?q={location}&units={unit}&appid={api_key}"
        
        # Send the request and get the response
        response = requests.get(url)
        
        # If the response is successful, parse the data
        if response.status_code == 200:
            data = response.json()
            # Extract the weather information
            temperature = data["main"]["temp"]
            weather_description = data["weather"][0]["description"]
            
            # Return the weather info in a dictionary
            return {
                "location": location,
                "temperature": temperature,
                "description": weather_description,
                "unit": unit
            }
        else:
            # Return an error message if the API request fails
            return {"error": "Unable to fetch weather data"}



# How it Works:

- **`__call__` method:** This method makes a request to the OpenWeatherMap API to fetch the weather data for a specified location and temperature unit (Celsius or Fahrenheit).
- **API Request:** The API request is formed using the location and the temperature unit passed to the method.
- **Response Handling:** If the response status code is 200 (successful), it extracts the temperature and weather description from the JSON data returned by OpenWeatherMap.
- **Returned Data:** The method returns a dictionary containing the location, temperature, weather description, and the unit.
- **Error Handling:** If the request fails, it returns an error message in a dictionary.

In [23]:
# Create an instance of the WeatherTool
weather_tool = WeatherTool()

# Fetch weather information for a location (e.g., "Dallas") in Fahrenheit
weather_info = weather_tool("Dallas", unit="fahrenheit")

# Print the result
print(weather_info)


{'location': 'Dallas', 'temperature': 287.53, 'description': 'few clouds', 'unit': 'fahrenheit'}


# Notebook Metadata

In [24]:
import platform
import sys
from datetime import datetime

# Display author information
author_name = "Huzaifa Irshad" 
github_username = "irshadhuzaifa"  

print(f"Author: {author_name}")
print(f"GitHub Username: {github_username}")

# Last modified datetime (file's metadata)
notebook_file = "Notebook_03_Setup_Custom_Toolkit.ipynb"
try:
    last_modified_time = os.path.getmtime(notebook_file)
    last_modified_datetime = datetime.fromtimestamp(last_modified_time)
    print(f"Last Modified: {last_modified_datetime}")
except Exception as e:
    print(f"Could not retrieve last modified datetime: {e}")

# Display platform, Python version, and Swarmauri version
print(f"Platform: {platform.system()} {platform.release()}")
print(f"Python Version: {sys.version}")

import swarmauri

try:
    version = swarmauri.__version__
except AttributeError:
    version = f"Swarmauri Version: 0.5.1"

print(f"Swarmauri Version: {version}")

Author: Huzaifa Irshad
GitHub Username: irshadhuzaifa
Last Modified: 2024-11-11 19:26:32.112423
Platform: Windows 11
Python Version: 3.12.7 | packaged by Anaconda, Inc. | (main, Oct  4 2024, 13:17:27) [MSC v.1929 64 bit (AMD64)]
Swarmauri Version: Swarmauri Version: 0.5.1
